A typical training procedure for a neural network is as follows:

+ Define the neural network that has some learnable parameters (or weights)
+ Iterate over a dataset of inputs
+ Process input through the network
+ Compute the loss (how far is the output from being correct)
+ Propagate gradients back into the network’s parameters
+ Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

`h/w = (h/w - kennel_size + 2 * padding) / stride + 1`

In [11]:
import torch
from torch import nn

connected_layer = nn.Linear(in_features=64*64*3, out_features=1)
input = torch.randn(1, 64, 64, 3)
input = input.view(1, 64*64*3)
print(input.shape)
output = connected_layer(input)
print(output.shape)

torch.Size([1, 12288])
torch.Size([1, 1])


In [12]:
import torch
import torch.nn as nn

x = torch.randn(10, 16, 30, 32) #batch, channel, height, width
print(x.shape)
cov = nn.Conv2d(16, 33, 3) #in_channel, out_channel, kennel_size
print(cov)
y = cov(x)
print(y.shape)

torch.Size([10, 16, 30, 32])
Conv2d(16, 33, kernel_size=(3, 3), stride=(1, 1))
torch.Size([10, 33, 28, 30])


In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3) #in_channel, out_channel, kennel_size
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16*6*6, 120) #in_features, out_features
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [38]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [3]:
for name, pa in net.named_parameters():
    print(name, ':', pa.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 3, 3])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 576])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [39]:
params[3].size()

torch.Size([16])

In [40]:
input = torch.randn(1, 1, 32, 32)  #batch, channels, height, width
out = net(input)
print(out)

tensor([[-0.0463, -0.0697,  0.0561, -0.0297,  0.0849, -0.0168,  0.0614, -0.1273,
          0.0695, -0.0012]], grad_fn=<AddmmBackward>)


In [41]:
out.shape

torch.Size([1, 10])

In [42]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [43]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1329, grad_fn=<MseLossBackward>)


In [44]:
loss.grad_fn

In [45]:
loss.grad_fn.next_functions[0][0]

In [46]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

In [47]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [48]:
loss.backward()
net.conv1.bias.grad

tensor([-0.0101, -0.0038,  0.0291, -0.0045, -0.0138, -0.0019])

`weight = weight - learning_rate * gradient`

In [49]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [50]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

In [51]:
net.conv1.bias.grad

tensor([-0.0086, -0.0096,  0.0143, -0.0040, -0.0105, -0.0072])

To backpropagate the error all we have to do is to `loss.backward()`. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [52]:
loss.item()

1.0986855030059814